## A simple model for demand and supply of publicly-provided services in a city

In [1]:
from enum import Enum
import os.path

import numpy as np
import pandas as pd
import geopandas as gpd
import geopy, geopy.distance
import shapely
from sklearn import gaussian_process

from matplotlib import pyplot as plt 
import seaborn as sns
plt.rcParams['figure.figsize']= (20,14)

In [2]:
from cityItems import AgeGroup, ServiceArea, ServiceType, SummaryNorm # enum classes for the model

In [3]:
gaussKern = gaussian_process.kernels.RBF
get_random_pos = lambda n: list(map(geopy.Point, list(zip(np.round(np.random.uniform(45.40, 45.50, n), 5), 
                                np.round(np.random.uniform(9.1, 9.3, n), 5)))))
make_shapely_point = lambda geoPoint: shapely.geometry.Point(tuple(geoPoint))

In [4]:
## ServiceUnit class
class ServiceUnit:
    def __init__(self, service, name='', position=geopy.Point(45.4641, 9.1919),
                 ageDiffusionIn={}, scaleIn=1, attributesIn={}):
        assert isinstance(position, geopy.Point), 'Position must be a geopy Point' 
        assert isinstance(service, ServiceType), 'Service must belong to the Eum'
        assert isinstance(name, str), 'Name must be a string'
        
        assert (np.isscalar(scaleIn)) & (scaleIn>0) , 'Scale must be a positive scalar'
        assert isinstance(attributesIn, dict), 'Attributes can be provided in a dict'
        
        self.name = name
        self.service = service
        
        # A ServiceType can have many sites, so each unit has its own. 
        # Moreover, a site is not uniquely assigned to a service
        self.site = position
        
        self.scale = scaleIn # store scale info
        self.attributes = attributesIn# dictionary
        
        # how the service availablity area varies for different age groups
        if ageDiffusionIn==None:
            self.ageDiffusion = {g: (1 + .005*np.round(np.random.normal(),2))*self.scale for g in AgeGroup.all()} 
        else:
            assert set(ageDiffusionIn.keys()) <= set(AgeGroup.all()), 'Diffusion keys should be AgeGroups'
            #assert all
            self.ageDiffusion = ageDiffusionIn
            
        # define kernel taking scale into account
        self.kernel = {g: gaussKern(length_scale=l*self.scale) for g, l in self.ageDiffusion.items()}
        
        
    def evaluate(self, targetPositions, ageGroup):
        assert all([isinstance(t, geopy.Point) for t in targetPositions]),'Geopy Points expected'
        
        # get distances
        distances = np.zeros(shape=(len(targetPositions),1))
        distances[:,0] = [geopy.distance.great_circle(x, self.site).km for x in targetPositions]
        
        # evaluate kernel to get level service score. If age group is not relevant to the service, return 0 as default
        if self.kernel.__contains__(ageGroup):
            score = self.kernel[ageGroup](distances, np.array([[0],]))
            # check conversion from tuple to nparray
            #targetPositions= np.array(targetPositions)
            #score2 = self.kernel[ageGroup](targetPositions, reshapedPos)
            #assert all(score-score2==0)
        else:
            score = np.zeros_like(targetPositions)
        return np.squeeze(score)
    
    @property
    def users(self): return list(self.propagation.keys())

In [5]:
### Supply modelling
def evaluate_services_at(positions, unitsList, outputServices= [t for t in ServiceType]):
    assert all([isinstance(t, geopy.Point) for t in positions]),'Geopy Points expected'
    # set all age groups as output default
    outputAgeGroups = AgeGroup.all()
    # initialise output
    outScores = {service: pd.DataFrame(np.zeros([len(positions), len(AgeGroup.all())]), 
                                 index=[tuple(t) for t in positions], columns=AgeGroup.all()) 
                 for service in outputServices}
    # loop over different services
    for thisServType in outputServices:
        serviceUnits = [u for u in unitsList if u.service == thisServType]
        if not serviceUnits:
            continue
        else:
            for thisAgeGroup in outputAgeGroups:
                unitValues = np.stack(list(map(lambda x: x.evaluate(positions, thisAgeGroup), serviceUnits)), axis=-1)
                # aggregate unit contributions according to the service type norm
                outScores[thisServType][thisAgeGroup] = thisServType.aggregate_units(unitValues)
            
    return outScores
        

In [6]:
test = [ServiceUnit(ServiceType.PoliceStation, 'Duomo', ageDiffusionIn=None), 
        ServiceUnit(ServiceType.PoliceStation, 'Ripamonti', 
                    position=geopy.Point(45.43, 9.201), ageDiffusionIn=None)]
evaluate_services_at(get_random_pos(4), test)


{<ServiceType.PoliceStation: (2, <ServiceArea.PublicSafety: 2>, <function SummaryNorm.<lambda> at 0x7f3abe824598>)>:                           AgeGroup.Newborn  AgeGroup.ChildPrimary  \
 (45.47922, 9.12771, 0.0)      9.680715e-07           9.248754e-07   
 (45.42492, 9.2677, 0.0)       1.212873e-06           1.252972e-06   
 (45.43049, 9.21085, 0.0)      7.444784e-01           7.450031e-01   
 (45.41904, 9.27348, 0.0)      5.944148e-08           6.185046e-08   
 
                           AgeGroup.ChildMid  AgeGroup.ChildHigh  \
 (45.47922, 9.12771, 0.0)       8.088207e-07        9.882940e-07   
 (45.42492, 9.2677, 0.0)        1.164358e-06        1.203029e-06   
 (45.43049, 9.21085, 0.0)       7.438204e-01        7.443470e-01   
 (45.41904, 9.27348, 0.0)       5.655040e-08        5.885273e-08   
 
                           AgeGroup.Young  AgeGroup.Junior  AgeGroup.Senior  \
 (45.47922, 9.12771, 0.0)    7.797157e-07     6.801834e-07     9.416824e-07   
 (45.42492, 9.2677, 0.0)     1.2

In [7]:
class UnitFactory:
    def __init__(self, path):
        assert os.path.isfile(path), 'File "%s" not found' % path
        self.filepath = path
        self.rawData = []
        
    def load(self):
        self.rawData = pd.read_csv(self.filepath, sep=';', decimal=',')
        self.nUnits = self.rawData.shape[0]
        defaultLocationColumns = ['Lat', 'Long']
        if set(defaultLocationColumns).issubset(set(self.rawData.columns)):
            print('Location data found')
            # store geolocations as geopy Point
            locations = [geopy.Point(self.rawData.loc[i, defaultLocationColumns]) for i in range(self.nUnits)]
            propertData = self.rawData.drop(defaultLocationColumns, axis=1)
        else:
            propertData = self.rawData
            locations = []
            
        return propertData, locations

    @staticmethod
    def createLoader(serviceType, path):
        if serviceType == ServiceType.School:
            return SchoolFactory(path)

In [8]:
class SchoolFactory(UnitFactory):
    
    def __init__(self, path):
        super().__init__(path)
        
    def load(self, meanRadius):
        
        assert meanRadius, 'Please provide a reference radius for the mean school size'
        (propertData, locations) = super().load()
        
        nameCol = 'DENOMINAZIONESCUOLA'
        typeCol = 'ORDINESCUOLA'
        scaleCol = 'ALUNNI'
        
        typeAgeDict = {'SCUOLA PRIMARIA': {AgeGroup.ChildPrimary:1},
                      'SCUOLA SECONDARIA I GRADO': {AgeGroup.ChildMid:1},
                      'SCUOLA SECONDARIA II GRADO': {AgeGroup.ChildHigh:1},}
        
        schoolTypes = propertData[typeCol].unique()
        assert set(schoolTypes) <= set(typeAgeDict.keys()), 'Unrecognized types in input'
        
        # set the scale to be proportional to the square root of number of children
        scaleData = propertData[scaleCol]**.5
        scaleData = scaleData/scaleData.mean() * meanRadius #mean value is mapped to input parameter
        propertData[scaleCol] = scaleData 
        unitList = []
                
        for scType in schoolTypes:
            bThisGroup = propertData[typeCol]==scType
            typeData = propertData[bThisGroup]
            typeLocations = [l for i,l in enumerate(locations) if bThisGroup[i]]

            for iUnit in range(typeData.shape[0]):
                rowData = typeData.iloc[iUnit,:]
                attrDict = {'level':scType}
                thisUnit = ServiceUnit(ServiceType.School, 
                        name=rowData[nameCol], 
                        position=typeLocations[iUnit], 
                        ageDiffusionIn=typeAgeDict[scType], 
                        scaleIn=rowData[scaleCol],
                        attributesIn=attrDict)
                unitList.append(thisUnit)
        
        return unitList

In [9]:
## Load scuole!
scuoleFile =  'final/milano_datiScuole.csv'
schoolLoader = UnitFactory.createLoader(ServiceType.School, scuoleFile)
schoolUnits = schoolLoader.load(0.5)

Location data found


In [45]:
## Aggregation
class UnitAggregator:
    '''
    A class to evaluate services on a grid and get average values for given land subdivisions
    '''
    
    def __init__(self, unitList, cityGeoFilesDict, gridStep=0.1): #gridStep in km
        assert isinstance(unitList, list), 'List expected'
        assert all([isinstance(t, ServiceUnit) for t in unitList]), 'ServiceUnits expected in list'
        self.units = unitList
        self.quartiereKey = 'quartieri' #hardcoded
        self.IDquartiereCol = 'ID_NIL'
        
        gridSize = 30
        
        # initialise values on grid
        self.serviceValues = {}
        
        # load division geofiles with geopandas
        self.subdivisions = {}
        for name, fileName in cityGeoFilesDict.items():
            self.subdivisions[name] = gpd.read_file(fileName)
            
        # compute city boundary
        self.cityBoundary = shapely.ops.cascaded_union(self.subdivisions[self.quartiereKey]['geometry'])
        
        # precompute coordinate ranges
        self.latRange = (self.cityBoundary.bounds[0], self.cityBoundary.bounds[2])
        self.longRange = (self.cityBoundary.bounds[1], self.cityBoundary.bounds[3])
        
        # precompute plotting grid
        (self.xPlot, self.yPlot) = np.meshgrid(np.linspace(*self.longRange, gridSize),
                                               np.linspace(*self.latRange, gridSize),
                                        indexing='ij')
        
        # construct point objects with same shape
        self.plotPoints = np.empty_like(self.xPlot, dtype=object)
        self.bInPerimeter = np.empty_like(self.xPlot, dtype=bool)
        self.IDquartiere = -1*np.ones_like(self.xPlot)
        
        for (i,j),_ in np.ndenumerate(self.plotPoints):
            self.plotPoints[i,j] = geopy.Point(self.yPlot[i,j], self.xPlot[i,j]) # (lat, long) format
            shplyPoint = make_shapely_point(self.plotPoints[i,j])
            # mark points outside boundaries
            self.bInPerimeter[i,j] = self.cityBoundary.contains(shplyPoint)
            if self.bInPerimeter[i,j]:
                bFound = False
                for iRow in range(self.subdivisions[self.quartiereKey].shape[0]):
                    thisQuartierePolygon = self.subdivisions[self.quartiereKey]['geometry'][iRow]
                    if thisQuartierePolygon.contains(shplyPoint):
                        # assign found ID
                        self.IDquartiere[i,j] = self.subdivisions[
                            self.quartiereKey][self.IDquartiereCol][iRow]
                        bFound = True
                        break # skip remanining zones
                assert bFound, 'Point within city boundary was not assigned to any zone'
        
    
    @property
    def longitude(self):
        return [unit.site.longitude for unit in self.units]
    @property
    def latitude(self):
        return [unit.site.latitude for unit in self.units]
    @property
    def scale(self):
        scales = [unit.scale for unit in self.units]
        return scales
    
    def compute_service_levels(self, servType):
        '''
        Evaluates on a grid the aggregate service level for the loaded ServiceUnits of a selected type.
        '''
        assert isinstance(servType, ServiceType), 'ServiceType expected in input'
        
        # extend internal cache with computed values
        self.serviceValues[servType] = evaluate_services_at(
            self.plotPoints.reshape(-1), self.units, [servType])[servType]
        return None
    
    @property
    def aggregate_values(self):
        
        assert self.serviceValues, 'Empty values on grid found, compute first please!'
        quartieri = self.subdivisions[self.quartiereKey][self.IDquartiereCol]
        # initialize
        out = {service: pd.DataFrame(np.zeros([len(quartieri), len(AgeGroup.all())]), 
                                 index=quartieri.values, columns=AgeGroup.all()) 
                 for service in self.serviceValues.keys()}
  
        for iRow in range(self.subdivisions[self.quartiereKey].shape[0]):
            thisQuartiere = self.subdivisions[self.quartiereKey].loc[iRow, self.IDquartiereCol]
            bThisQuartiere = self.IDquartiere == thisQuartiere
            # loop over service types
            for servType in self.serviceValues.keys():
                for ageGroup, valuesSeries in self.serviceValues[servType].items(): 
                    meanValue = np.vectorize(lambda p: np.mean(valuesSeries[tuple(p)]))(
                        self.plotPoints[bThisQuartiere])
                    out[servType].loc[thisQuartiere, ageGroup] = meanValue
                    
        return out

In [49]:
test = UnitAggregator(schoolUnits, {'quartieri':'data/milanoNIL.geojson'})
test.compute_service_levels(ServiceType.School)

In [50]:
test.aggregate_values

ValueError: cannot call `vectorize` on size 0 inputs unless `otypes` is set

In [ ]:
nil = gpd.read_file('data/milanoNIL.geojson')
#xx = nil['geometry'][0]
#shapely..union(nil['geometry'][1])
zz = shapely.ops.cascaded_union(nil['geometry'])
zz.bounds

In [ ]:
nil

In [ ]:
nil = gpd.read_file('data/milanoNIL.geojson')
#xx = nil['geometry'][0]
#shapely..union(nil['geometry'][1])
cityBounds = shapely.ops.cascaded_union(nil['geometry'])
IDquartiereCol = 'ID_NIL'

gridSize = 40

# precompute coordinate ranges
latRange = (cityBounds.bounds[0], cityBounds.bounds[2])
longRange = (cityBounds.bounds[1], cityBounds.bounds[3])
        
# precompute plotting grid
(xPlot, yPlot) = np.meshgrid(np.linspace(*longRange, gridSize),
                                               np.linspace(*latRange, gridSize),
                                        indexing='ij')

plotPoints = np.empty_like(xPlot, dtype=object)
bInPerimeter = np.empty_like(xPlot, dtype=bool)
quartiereID = -1*np.ones_like(xPlot) # initialize to -1

for (i,j),_ in np.ndenumerate(plotPoints):
            plotPoints[i,j] = geopy.Point(yPlot[i,j], xPlot[i,j]) # (lat, long) format
            # mark points outside boundaries
            shplyPoint = make_shapely_point(plotPoints[i,j])
            bInPerimeter[i,j] = cityBounds.contains(shplyPoint)
            if bInPerimeter[i,j]:
                bFound = False
                for iRow in range(nil.shape[0]):
                    if nil['geometry'][iRow].contains(shplyPoint):
                        # assign found ID
                        quartiereID[i,j] = nil[IDquartiereCol][iRow]
                        bFound = True
                        break # skip remanining zones
                assert bFound, 'Point within city boundary was not assigned to any zone'
        
# mark points outside boundaries
#tt = plotPoints[0,0]
#bInPerimeter = zz.contains(shapely.geometry.Point(plotPoints[0,0].longitude, plotPoints[0,0].latitude))


ww = [cityBounds.contains(make_shapely_point(p)) for p in plotPoints.flat]
bInBoundary = np.array(ww).reshape(xPlot.shape)
#make_shapely_point = lambda geoPoint: shapely.geometry.Point(tuple(geoPoint))
#zz.y


In [ ]:
bInBoundary = np.array(ww).reshape(xPlot.shape)
bInBoundary.mean()

plt.scatter(xPlot[bInBoundary], yPlot[bInBoundary], s=quartiereID[bInBoundary])
plt.show()

In [ ]:
## Plot tools
class unitPlotter:
    '''
    A class that plots various types of output from a list of ServiceUnits
    '''
    def __init__(self, unitList, gridSize=40):
        assert isinstance(unitList, list), 'List expected'
        assert all([isinstance(t, ServiceUnit) for t in unitList]), 'ServiceUnits expected in list'
        self.units = unitList
        self.gridSize = gridSize
        # precompute coordinate ranges
        self.latRange = (min(self.latitude), max(self.latitude))
        self.longRange = (min(self.longitude), max(self.longitude))
        
        # precompute plotting grid
        (self.xPlot, self.yPlot) = np.meshgrid(np.linspace(*self.longRange, gridSize),
                                               np.linspace(*self.latRange, gridSize),
                                        indexing='ij')
        # construct point objects with same shape
        self.plotPoints = np.empty_like(self.xPlot, dtype=object)
        for (i,j),_ in np.ndenumerate(self.plotPoints):
            self.plotPoints[i,j] = geopy.Point(self.yPlot[i,j], self.xPlot[i,j]) # (lat, long) format
        
        self.serviceValues = {}
    
    @property
    def longitude(self):
        return [unit.site.longitude for unit in self.units]
    @property
    def latitude(self):
        return [unit.site.latitude for unit in self.units]
    @property
    def scale(self):
        scales = [unit.scale for unit in self.units]
        return scales
    
    
    def plot_locations(self):
        '''
        Plots the loaded ServiceUnits according to their locations and rescaling the relative sizes
        '''
        plotScales = self.scale/np.mean(self.scale)
        plt.figure()
        plt.scatter(self.latitude, self.longitude, s=plotScales)
        plt.axis('equal')
        plt.show()
        
    def compute_service_levels(self, servType):
        '''
        Evaluates on a grid the aggregate service level for the loaded ServiceUnits of a selected type.
        '''
        assert isinstance(servType, ServiceType), 'ServiceType expected in input'
        
        # extend internal cache with computed values
        self.serviceValues[servType] = evaluate_services_at(
            self.plotPoints.reshape(-1), self.units, [servType])[servType]
        return None
        
    def plot_service_levels(self, servType):
        '''
        Plots a contour graph of the results for each ageGroup.
        '''
        assert isinstance(servType, ServiceType), 'ServiceType expected in input'
        for ageGroup, valuesSeries in self.serviceValues[servType].items():
            valuesArray = np.vectorize(lambda p: valuesSeries[tuple(p)])(self.plotPoints)
            plt.figure()
            plt.title(ageGroup)
            CS = plt.contourf(self.xPlot, self.yPlot, valuesArray)
            cbar = plt.colorbar(CS)
            cbar.ax.set_ylabel('Service level')
            plt.show()
            
        return None

In [ ]:
plotter = unitPlotter(schoolUnits, gridSize=40)
plotter.compute_service_levels(ServiceType.School)

plotterLight = unitPlotter(schoolUnits, gridSize=16)
plotterLight.compute_service_levels(ServiceType.School)

In [ ]:
sns.distplot(plotter.scale,50)

In [ ]:
plotter.plot_service_levels(ServiceType.School)

In [ ]:
import geopandas as gpd


In [ ]:
class zoneIntegrator:

    def __init__(self, unitList, divisionFilesDict):
        assert isinstance(unitList, list), 'List expected'
        assert all([isinstance(t, ServiceUnit) for t in unitList]), 'ServiceUnits expected in list'
        self.units = unitList
        
        # load division geofiles with geopandas
        self.divisions = {}
        for name, fileName in divisionFilesDict.items():
            self.divisions[name] = gpd.read_file(fileName)
        
    def compute_means(self):
        quartiereKey = 'quartieri'
        for iRow in range(self.divisions[quartiereKey].shape[0]):
            thisPolygon = self.divisions[quartiereKey].loc[iRow, 'geometry']
            gridPoints = np.random.uniform(thisPolygon.bounds)
            print(thisPolygon)


In [ ]:
from shapely.geometry import Point
def grid_points_within(poly, nGrid=100:
    min_x, min_y, max_x, max_y = poly.bounds
    
    while len(points) < num_points:
        random_point = Point([np.random.uniform(min_x, max_x), np.random.uniform(min_y, max_y)])
        if (random_point.within(poly)):
            points.append(random_point)

    return points

In [ ]:
thisPolygon = zint.divisions['quartieri'].loc[0, 'geometry']
testPoints = random_points_within(thisPolygon,2)

xx =  random_points_within(thisPolygon,20)
for t in xx:
    plt.plot(t)
plt.plot(thisPolygon)
plt.show()
#[geopy.Point(p.y, p.x) for p in xx]
#xx

In [ ]:
zint = zoneIntegrator(schoolUnits, {'quartieri':'data/milanoNIL.geojson'})
#zint.divisions['quartieri'].plot()
zint.compute_means()


In [ ]:
zz = zint.divisions['quartieri']
zz

## Demand modelling

In [ ]:
polySample = zz['geometry'][86]
ww = np.asarray(polySample.exterior.coords)

In [ ]:
### Demand modelling
class Household:
    def __init__(self, position=None, membersInput=None):
        # make defaults
        if not position: position = get_random_pos(1)
        if not membersInput: membersInput = {a: 1 for a in AgeGroup.all()}
        # expand input to all age group keys
        self.members = {a: membersInput.get(a, 0) for a in AgeGroup.all()}
        self.position = position
        self.export = pd.DataFrame(self.members, index=([self.position])) # precompute for speed

def evaluate_demand(householdList, outputServices= [t for t in ServiceType]):
    """ """
    # initialise output
    outDemand = dict()
    # consolidate positions. If two households share the same position, sum components.
    householdData = pd.concat([h.export for h in householdList])
    householdData['position'] = householdData.index 
    consolidated = householdData.groupby('position').sum()
    
    for thisServType in outputServices:
        outDemand[thisServType] = consolidated*thisServType.demandFactors
        
    return outDemand

In [ ]:
hhList =  [Household() for i in range(40)]
evaluate_demand(hhList)

In [ ]:
## Matching demand and supply
def get_satisfaction_indexes(householdList)